# Конечные автоматы

#### Математика для лингвистов
#### Клышинский Э.С., 09.10.2020

### Теоретическое введение


**Абстрактный автомат** (в теории алгоритмов) — математическая абстракция, модель дискретного устройства, имеющего один вход, один выход и в каждый момент времени находящегося в одном состоянии из множества возможных. На вход этому устройству поступают символы одного алфавита, на выходе оно выдаёт символы (в общем случае) другого алфавита.

Формально абстрактный автомат определяется как пятёрка

A = ( S , X , Y , δ , λ ) 

Где S — конечное множество состояний автомата,<br> X, Y — конечные входной (терминальный) и выходной алфавиты соответственно, из которых формируются строки, считываемые и выдаваемые автоматом,<br> δ : S × X → S — функция переходов,<br> λ : S × X → Y — функция выходов. 

**Конечный автомат** (КА) — (в теории алгоритмов) — математическая абстракция, модель дискретного устройства, имеющего один вход, один выход и в каждый момент времени находящегося в одном состоянии из множества возможных. Является частным случаем абстрактного дискретного автомата, число возможных внутренних состояний которого конечно. 

Помимо конечных автоматов существуют и бесконечные дискретные автоматы — автоматы с бесконечным числом внутренних состояний, например, **машина Тьюринга**. 

### Представление и использование конечного автомата в виде графа

Конечный автомат может использоваться в двух основных случаях:
* управление технической системой;
* проверка принадлежности строки некоторому языку.

Конечный автомат можно представить в виде графа, в котором состояния будут представлены вершинами, дуги будут показывать разрешенные переходы из одного состояния в другое. Дуги будут помечены символами входного алфавита и показывать, по каким входным символам разрешен переход.  Также дуги могут быть помечены символами выходного алфавита и будут показывать, какой необходимо произвести выход при переходе по данной дуге.

<img src="img/fsm_enemy_brain.png">
(c) <a href="https://tproger.ru/translations/finite-state-machines-theory-and-implementation/">Typical Proger & Authors</a>

При разборе строки также определяются конечные состояния конечного автомата. Если разбор строки был завершен в конечном состоянии, то строка считается корректной. В противном случае строка трактуется как некорректная в рамках данного языка.

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/%D0%93%D1%80%D0%B0%D1%84_%D0%BF%D0%B5%D1%80%D0%B5%D1%85%D0%BE%D0%B4%D0%BE%D0%B2_%D0%94%D0%9A%D0%90.svg/1280px-%D0%93%D1%80%D0%B0%D1%84_%D0%BF%D0%B5%D1%80%D0%B5%D1%85%D0%BE%D0%B4%D0%BE%D0%B2_%D0%94%D0%9A%D0%90.svg.png" width=50%>

Конечные автоматы позволяют разбирать простые языки, описываемые правилами двух видов:
* $A \rightarrow \gamma B$
* $A \rightarrow \gamma$,
где $\gamma$ - символ из входного (терминального) словаря (терминал).

По первому правилу если в текущей позиции стоит символ $\gamma$, то происходит переход в состояние B. По второму правилу после символа $\gamma$ разбор прекращается (происходит переход в некоторое состояние, которое является конечным и не имеет выходных дуг).

Заметим, что регулярные грамматики являются самыми простыми. Например, они не смогут корректно разобрать выражение, в котором скобки сбалансированы: (\[\{ \}\{ \}\]\[ \]\[\{ \}\]\)\( \) . Но при этом легко справятся с разбором идентификатора при помощи следующей грамматики (здесь маленькие буквы относятся к терминальному алфавиту, а большие - к алфавиту состояний, $\epsilon$ - пустой символ, многоточие означает, что мы пропустили часть правил и не является частью языка описания правил):
* $A \rightarrow \_A | aB | bB | ... | zB $ 
* $B \rightarrow \_B | aB | bB | ... | zB | 0B | ... | 9B \epsilon$ 

Здесь мы считаем, что идентификатор может состоять только из букв и знаков подчеркивания, букв и цифр, причем цифры не могут идти после знаков подчеркивания, идентификатор не может состоять только из знаков подчеркивания.

Тогда идентификатор `__public` может быть выведен как следующая последовательность замен:
* $A \xrightarrow{A\rightarrow \_A} \_A \xrightarrow{A\rightarrow \_A} \_\_A \xrightarrow{A\rightarrow pB} \_\_pB \xrightarrow{B\rightarrow uB} \_\_puB \xrightarrow{B\rightarrow bB} \_\_pubB  \xrightarrow{B\rightarrow lB} \_\_publB  \xrightarrow{B\rightarrow iB} \_\_publiB  \xrightarrow{B\rightarrow cB} \_\_publicB  \xrightarrow{B\rightarrow \epsilon} \_\_public $ 


### Хранение конечного автомата

Конечный автомат можно представить в виде графа или матрицы, а можно просто записать в коде. Начнем с последнего варианта. Запишем функцию для разбора идентификаторов.

In [4]:
# Разбор идентификатора в коде.
def isIdentifier(line):
    state = 'A' # Начальное состояние - А.
    correct = True
    for s in line:
        if state == 'A': 
            if s == '_':
                state = 'A'
            elif s >= 'a' and s <= 'z':
                state = 'B'
            else:
                correct = False
                break
        if state == 'B': 
            if s == '_' or (s >= 'a' and s <= 'z') or (s >= '0' and s <= '9'):
                state = 'B'
            else:
                correct = False
                break
    return correct and state == 'B'

In [6]:
print(isIdentifier('_asd'))
print(isIdentifier('asd12'))
print(isIdentifier('asd'))
print(isIdentifier('_'))
print(isIdentifier('123'))
print(isIdentifier('123d'))

True
True
True
False
False
False


Теперь создадим матрицу, описывающую переходы между состояниями. Первая колонка матрицы будет обозначать переход по символу '\_', вторая колонка - по букве, третья - по цифре. Состояния обозначим их номерами. Состояние -1 будет означать ошибку. Последняя колонка - является ли состояние конечным.

In [12]:
transitions[0]

[0, 1, -1]

In [19]:
transitions = [[0, 1, -1],[1, 1, 1]]

def char2code(s):
    if s == '_':
        return 0
    elif s >= 'a' and s <= 'z':
        return 1
    elif s >= '0' and s <= '9':
        return 2    

def isIdentifier2(line):
    state = 0
    correct = True
    for s in line:
        state = transitions[state][char2code(s)]
        if state == -1:
            correct = False
            break
    return correct and state == 1
        

In [20]:
print(isIdentifier2('_asd'))
print(isIdentifier2('asd12'))
print(isIdentifier2('asd'))
print(isIdentifier2('_'))
print(isIdentifier2('123'))
print(isIdentifier2('123d'))

True
True
True
False
False
False


Напишем теперь конечный автомат для решения следующей задачи. Необходимо разобрать строку, содержащую арифметическое выражение без скобок. Необходимо для каждого идентификатора выдать строку "id", а для каждого арифметического оператора - "sign". Но так как до прихода знака мы не можем сказать, что идентификатор закончился, то будем на знаки (кроме унарного минуса) выдавать "id, sign".

Создадим еще одну матрицу, аналогичную матрице переходов, но содержащую выдачу для каждого перехода. Добавим информацию о том, является ли состояние конечным.

Расширим наш алфавит знаками арифметических операций. Учтем, что унарный минус также может встретиться в выражении. Наконец, добавим целые числа.

In [33]:
transitions3 = [[1, 2, -1, -1, 3], [1, 2, -1, -1, -1], [2, 2, 2, 1, 1], [1, 2, -1, -1, -1]]
final = [0, 0, 1, 0]
output = [['','','','','sign'], ['','','','',''],['','','','id, sign', 'id, sign'],['','','','','']]

def char2code3(s):
    if s == '_':
        return 0
    elif s >= 'a' and s <= 'z':
        return 1
    elif s >= '0' and s <= '9':
        return 2
    elif s in ['+', '*', '/']:
        return 3
    elif s == '-':
        return 4

def isIdentifier3(line):
    state = 0
    correct = True
    for s in line:
        out = output[state][char2code3(s)]
        state = transitions3[state][char2code3(s)]
        if out != '':
            print("-> ", out)
        if state == -1:
            correct = False
            break
    if correct and final[state] == 1:
        print("-> id")
    return correct and final[state] == 1
        

In [34]:
print(isIdentifier3('_asd'))
print(isIdentifier3('asd12'))
print(isIdentifier3('asd'))
print(isIdentifier3('asd+asd'))
print(isIdentifier3('-_asd*dsw_sd'))
print(isIdentifier3('_'))
print(isIdentifier3('123'))
print(isIdentifier3('123d'))

-> id
True
-> id
True
-> id
True
->  id, sign
-> id
True
->  sign
->  id, sign
-> id
True
False
False
False



### Детерминированные и недетерминированные конечные автоматы

Автомат называется детерминированным (ДКА), если на каждом шаге терминал из входной цепочки однозначно определяет следующее текущее состояние. Его противоположность – недетерминированный конечный автомат (НКА), в котором из одной вершины может выходить несколько дуг, помеченных одним символом, либо присутствуют е-дуги.

Минусом НКА является то, что мы не знаем куда перейти из текщей вершины, еслси текущим символом помечены более, чем одна дуга. Если мы выберем оба маршрута, то дальше нам снова придется встать перед выбором в другом состоянии. Из-за этого сложность разбора будет расти экспоненциально.

Если конечный автомат содержит в себе переходы по ε, к нему может быть применен

#### Алгоритм избавления от е-дуг

Е-дугой называется дуга, помеченная пустым символом ε. Символ ε позволяет упросить конечный автомат, однако приводит к тому, что мы не знаем, следует ли перейти по дуге, помеченной очередным символом, или следует выбрать е-дугу.

Для того, чтобы избавиться от е-дуги воспользуемся свойством эквивалентности правой и левой части правила и возможностью заменить одно на другое. В примере ниже вхождение B в правую часть продукции может быть заменено на свою правую часть. Однако α ε = α. Если теперь переобозначить B$\rightarrow$γ, то получится, что мы избавились от ε.

<table>
    <tr><td bgcolor="#FFFFFF">A$\rightarrow$αB|β </td><td bgcolor="#FFFFFF"> $\Leftrightarrow$ </td><td bgcolor="#FFFFFF"> A$\rightarrow$αB| α |β </td><td bgcolor="#FFFFFF"> $\Leftrightarrow$ </td><td bgcolor="#FFFFFF">A$\rightarrow$αγ| α |β </td></tr>
    <tr><td style="text-align:left">B$\rightarrow$γ|ε </td><td></td><td style="text-align:left"> B$\rightarrow$γ </td><td></td></tr>
</table>

### Генерация НКА по регулярному выражению

Создадим начальную и конечную вершины графа конечного автомата. Теперь наша задача соединить эти вершины по регулярному выражению. Создадим дугу, помеченную этим регулярным выражениям. Теперь для всех дуг графа, содержащих цепочки, более сложные, чем единственный терминал, применим следующие правила.

* Для дуги, соединяющей вершины S и Z, помеченной цепочкой αβ создадим промежуточную вершину T и разделим цепочку следующим образом. $S\xrightarrow{αβ}Z \Rightarrow S\xrightarrow{α}T\xrightarrow{β}Z$
* Для дуги, соединяющей вершины S и Z, помеченной цепочкой α|β создадим две параллельные дуги. $S\xrightarrow{αβ}Z \Rightarrow S \xrightarrow{α}Z, S\xrightarrow{β}Z$
* Для дуги, соединяющей вершины S и Z, помеченной цепочкой $α^*$ создадим промежуточную вершину Т и три дуги. $S\xrightarrow{αβ}Z \Rightarrow S \xrightarrow{ε}T, T\xrightarrow{ε}Z, T\xrightarrow{α}T$
* Для дуги, соединяющей вершины S и Z, помеченной цепочкой $α^+$ создадим промежуточную вершину Т и три дуги. $S\xrightarrow{αβ}Z \Rightarrow S \xrightarrow{α}T, T\xrightarrow{ε}Z, T\xrightarrow{α}T$

#### Пример генерации НКА по регулярному выражению
1) <img src="img/FA1.png">
2) <img src="img/FA2.png">
3) <img src="img/FA3.png">
4) <img src="img/FA4.png">
5) <img src="img/FA5.png">
6) <img src="img/FA6.png">

### Методы оптимизации КА

#### Избавление от е-дуг

Е-дуга показывает, что мы можем перейти из одного состояния в другое без перемещения по входной строке. Того же эффекта можно добиться копированием всех дуг, исходящих из дуги, в которую входит е-дуга, так, чтобы они начинали в состоянии, из которого исходит е-дуга, и заканчивались в том же состоянии, в котором они заканчивались до копирования. Если е-дуга входит в конечное состояние, то состояние, из которого она исходит, тоже становится конечным.
<img src="img/FA7.png">
После замены е-дугу можно удалить.
<img src="img/FA8.png">

#### Избавление от недостижимых состояний

Недостижимыми называются состояния, в которые нельзя попасть по какому-либо пути из начального. Для того, чтобы избавиться от недостижимых символов, используют следующий алгоритм.
1. Внесем в некоторое множество начальный символ.
2. Добавим в множество все состояния, достижимые из состояний, входящих в это множество.
3. Будем повторять шаг 2 до тех пор, пока на нем будут добавляться вершины.

<img src="img/FA9.png">

#### Устранение эквивалентных состояний
 
В КА некоторые состояния могут оказаться эквивалентными, то есть при получении одних и тех же терминалов в них производятся переходы в одни и те же или эквивалентные состояния, при этом оба состояния должны быть оба конечными или не конечными.

1. Множество всех состояния разбивается на множество конечных и не конечных символов.
2. Если по одним и тем же терминалам два различных состояния подмножества переходят в одно и то же подмножество, то они остаются в этом подмножестве. В противном случае они выделяются в различные подмножества.
3. Шаг 2 производится до тех пор, пока разбиение на подмножества возможно.



### Преобразование НКА в ДКА

1. Удаляем е-дуги НКА если они есть.
2. Создаем начальное состояние ДКА, представляющее собой объединение всех начальных состояний НКА.
3. Для всех ячеек ДКА проверяем, имеется ли в ДКА состояние, в которое должен осуществляться переход. При отсутствии такого состояния создаем его путем объединения вершин НКА, имена которых записаны в данной ячейке.
4. Удаляем полученные эквивалентные состояния.

<img src="img/FA10.png">